![](./images/09_02.jpg)

**Tóm tắt bài toán**
* MỘt công ty công nghệ bi hack và họ đã lưu lại thông tin của các hacker mà chúng để lại, người ta mong muốn dựa trên các thông tin này để biết dc liệu có 2 hay 3 hacker tiến hành hack công ty này trong một phi vụ. Các feature bao gồm:
  * `Session_Connection_Time`: thời gian một session duy trì (tính theo phút)
  * `Bytes_Transferred`: số bytes lấy đi trong một session
  * `Kali_Trace_Used`: có sử dụng các dịch vụ của Kali Linux hay ko
  * `Servers_Corrupted`: số server bị hỏng trong quá trình tấn công
  * `Pages_Corrupted`: số trang bị truy cấp bất hợp pháp
  * `Location`: địa chỉ của hacker (ko quan trọng nếu hacker sử dụng dịch vụ VPN)
  * `WPM_Typing_Speed`: tốc độ đánh máy của hacker, tức số từ trong một session
* Người ta thấy rằng trong 100 cuộc tấn công, thì các hacker sẽ chia đều nhiệm vụ cho nhau, giả sư 2 hacker thì mỗi hacker sẽ làm 50 cuộc tấn công, 3 hacker thì mỗi hacker sẽ đảm nhiệm 33 cuộc tấn công. Hãy build model xem có 2 hay 3 hacker tham gia tấn công công ty công nghệ này.

# 1. Đọc dữ liệu

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('hack_find').getOrCreate()

In [3]:
dataset = spark.read.csv('./data/hack_data.csv', inferSchema=True, header=True)

In [4]:
dataset.show(3, False)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|Location              |WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+
|8.0                    |391.09           |1              |2.96             |7.0            |Slovenia              |72.37           |
|20.0                   |720.99           |0              |3.04             |9.0            |British Virgin Islands|69.08           |
|31.0                   |356.32           |1              |3.71             |8.0            |Tokelau               |70.58           |
+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+
only showing top 3 rows



In [5]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [6]:
dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



# 2. Chuyển đổi dữ liệu

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
feat_cols = dataset.columns.copy()

In [9]:
feat_cols.remove('Location')

In [10]:
print(feat_cols)

['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed']


In [11]:
vec_assembler = VectorAssembler(inputCols=feat_cols, outputCol='features')

In [12]:
final_data = vec_assembler.transform(dataset)

In [13]:
final_data.show(3, False)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+--------------------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|Location              |WPM_Typing_Speed|features                        |
+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+--------------------------------+
|8.0                    |391.09           |1              |2.96             |7.0            |Slovenia              |72.37           |[8.0,391.09,1.0,2.96,7.0,72.37] |
|20.0                   |720.99           |0              |3.04             |9.0            |British Virgin Islands|69.08           |[20.0,720.99,0.0,3.04,9.0,69.08]|
|31.0                   |356.32           |1              |3.71             |8.0            |Tokelau               |70.58           |[31.0,356.32,1.0,3.71,8.0,70.58]

# 3. Scale dữ liệu

In [14]:
from pyspark.ml.feature import StandardScaler

In [15]:
scaler = StandardScaler(inputCol="features",
                        outputCol="scaled_features",
                        withStd=True,
                        withMean=False)

In [16]:
scaler_model = scaler.fit(final_data)

In [17]:
cluster_final_data = scaler_model.transform(final_data)

# 4. Build model
* Vì người ta chỉ cần biết có 2 hay 3 hacker tham gia tấn công tên $k-value \in \{2, 3\}$ là okla

In [18]:
from pyspark.ml.clustering import KMeans

In [19]:
kmeans2 = KMeans(featuresCol='features', k=2)
kmeans3 = KMeans(featuresCol='features', k=3)

In [20]:
model_k2 = kmeans2.fit(cluster_final_data)
model_k3 = kmeans2.fit(cluster_final_data)

* Tiến hành dự đoán

In [21]:
prediction_k2 = model_k2.transform(cluster_final_data)

In [22]:
prediction_k3 = model_k3.transform(cluster_final_data)

# 5. Đánh giá model - tính silhouette coefficient

In [23]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [24]:
elevator = ClusteringEvaluator()

In [25]:
silhouette_k2 = elevator.evaluate(prediction_k2)
silhouette_k3 = elevator.evaluate(prediction_k3)

In [26]:
silhouette_k2, silhouette_k3

(0.8048521975748283, 0.8048521975748283)

> **Nhận xét**
> * Silhouette của $k-value = 2$ và $k-value = 3$ bằng nhau, vậy tức chỉ có duy nhất 2 cluster thôi

In [27]:
prediction_k2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  170|
|         0|  164|
+----------+-----+



In [28]:
prediction_k3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  170|
|         0|  164|
+----------+-----+



> **Kết luận**:
> Chỉ có 2 hacker